# OCR on SageMaker--推理

## 0 运行环境
本文在boto3 1.15.16和sagemaker 2.15.0下测试通过。

In [ ]:
import boto3,sagemaker
print(boto3.__version__)
print(sagemaker.__version__)

如果版本较低，可运行下行代码进行升级，升级完毕后，需要重启Kernal，然后再检查版本

In [ ]:
!pip install boto3 --upgrade -i https://opentuna.cn/pypi/web/simple/
!pip install sagemaker --upgrade -i https://opentuna.cn/pypi/web/simple/

### 1 获取image

本项目已build完毕image，存放到ECR中，可直接部署到SageMaker。请选择选择合适版本。

In [ ]:
#使用CPU版本
image_uri="048912060910.dkr.ecr.cn-northwest-1.amazonaws.com.cn/nwcd/ocr-inference:2.0.2-2.0-cpu"

In [ ]:
#使用GPU版本
image_uri="048912060910.dkr.ecr.cn-northwest-1.amazonaws.com.cn/nwcd/ocr-inference:2.0.2-2.0-gpu"

## 2 部署到SageMaker
CPU推理目前只能使用large机型，其他机型会报`Intel MKL function load error: cpu specific dynamic library is not loaded.`

In [ ]:
%%time
from sagemaker.model import Model
import boto3

iam = boto3.client('iam')
roles = iam.list_roles(PathPrefix='/service-role')
role=""
for current_role in roles["Roles"]:
    if current_role["RoleName"].startswith("AmazonSageMaker-ExecutionRole-"):
        role=current_role["Arn"]
        break
#如果role为空表示有问题
print("ExecutionRole:"+role)

endpoint_name = "ocr"
my_model = Model(role=role,image_uri=image_uri)

#该步骤,大概需要7-8分钟；根据需要修改instance_type
xgb_predictor = my_model.deploy(initial_instance_count=1,
                                endpoint_name=endpoint_name,
                                instance_type='ml.m5.large'
                                )

## 3 推理

### 准备测试图片

In [ ]:
data = {"bucket":"nowfox","image_uri":["data/test/008.jpg"]}

返回的结果为textract格式

### 方式一、使用boto3

In [ ]:
import boto3
import json
endpoint_name = "ocr"
client = boto3.client('runtime.sagemaker')

response = client.invoke_endpoint(EndpointName=endpoint_name,
                                  Body=json.dumps(data),
                                  ContentType="application/json")
#print(response)
response_body = response['Body'] 
body= response_body.read()
result = json.loads(body.decode('utf-8'))
print(result)

### 方式二、使用SageMaker SDK

In [ ]:
import sagemaker
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
predictor = sagemaker.predictor.Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sagemaker.Session(),
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer())
result = predictor.predict(data)
print(result)

## 4 删除Endpoint

In [ ]:
import boto3
client = boto3.client(service_name='sagemaker') 
client.delete_endpoint(EndpointName=endpoint_name)
client.delete_endpoint_config(EndpointConfigName=endpoint_name)